In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
%autosave 0

Autosave disabled


# WeightWatcher Research

> WeightWatcher (w|w) is an open-source, diagnostic tool for analyzing Deep Neural Networks (DNN),
> without needing access to training or even test data. It is based on theoretical research into
> Why Deep Learning Works, based on our Theory of Heavy-Tailed Self-Regularization (HT-SR)... <sup>[1]</sup>

## Objectives

1. How well does the WeightWatcher metric "alpha" work when comparing
    - different variants of different architecture?
    - different variants of the same architecture?
    - differently trained models of the same variant and architecture?
2. What are reasonable explanations for those observations?

## Background

The main contribution of the WeightWatcher tool is the unique metric "alpha".
It is...

## Analysis

### Internal services

First off, instantiate some internal services.
- `WeightWatcherService`: Simple wrapper for the functionality provided by the actual `weightwatcher` module
- `WeightWatcherResultService`: Responsible for storing and loading WeightWatcher results
- `AnalysisService`: Functionality for further analysis of the WeightWatcher results

In [2]:
# Instantiate necessary services

from weight_watcher import WeightWatcherResultService
from visualization import PlottingService, DashboardService

plotting_service = PlottingService()
dashboard_service = DashboardService()
analysis_result_repository = WeightWatcherResultService('results')

TypeError: __init__() got an unexpected keyword argument 'unbound_message'

In [ ]:
# Create plots

from typing import List
import plotly.graph_objects as go

results = analysis_result_repository.load_all()

plots: List[go.Figure] = []
plots += [plotting_service.create_summaries_plot(results)]
plots += plotting_service.create_summaries_per_architecture_plot(results)
plots += [plotting_service.create_details_plots(results)]
plots += plotting_service.create_details_per_architecture(results)
plots += plotting_service.create_details_per_model(results)

dashboard_service.build_dashboard(plots)
dashboard_service.show_dashboard()

## References

[1]: <cite>[WeightWatcher Homepage](https://weightwatcher.ai/#Research)</cite>
[2]: <cite>[WeightWatcher Homepage](https://weightwatcher.ai/#Research)</cite>